![](./data/mdjn.png)
input: multimodal diffusion, ai, latent space, text input, image output

# Что нам нужно?

* Метод, при помощи которого мы будем генерировать какое-то не существующее до этого изображение ([forvard/reverse diffusion](https://arxiv.org/abs/2006.11239))
* Способ соединить вместе текст и изображение ([text-image representation model](https://arxiv.org/abs/2103.00020))
* Что-то для сжатия изображений (SD is a LDM)([autoencoder](https://arxiv.org/abs/2112.10752))
* Способ задать направление для генерации ([U-net + attention](https://arxiv.org/abs/2112.10752))

![](./data/breaf_scheme.png)

## Метод, при помощи которого мы будем генерировать какое-то не существующее до этого изображение
В предыдущем докладе данного юнита мы подробно останавливались на том, что такое диффузия, и какой она бывает.
В связи с этим далее будет картинка-плэйсхолдер, которая должна освежить память слушателя. (A если этого не произойдёт, к предыдущему докладу всегда можно [вернуться](./data/from_scratch.ipynb))

![](./data/ddpm_plot.png)

## Способ соединить вместе текст и изображение
CLIP - Contrastive Language-Image Pre-training
![](./data/CLIP_scheme.png)

(1) В первой части схемы показан принцип дообучения энкодера для изображений (например, ResNet50 или ViT) и энкодера для текстов (GPT-like трансформер) в Contrastive стратегии для батча размера N. Из-за того, что используемое расстояние не симметрично, "расталкивать" представления нужно в обе стороны, что выливается в соответствие поставленной задачи минимизации следующей функции потерь:

$$\Large \ell^{(I\rightarrow T)}_i = -log\frac{e^{\frac{\langle I_i, T_i\rangle}{τ}}}{\sum_{k=1}^{N} e^{\frac{\langle I_i, T_k\rangle}{τ}}};$$




$$\Large \ell^{(T\rightarrow I)}_i = -log\frac{e^{\frac{\langle T_i, I_i\rangle}{τ}}}{\sum_{k=1}^{N} e^{\frac{\langle T_i, I_k\rangle}{τ}}}$$

$$\Large \mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} (\lambda \ell^{(I\rightarrow T)}_i + (1 - \lambda) \ell^{(T\rightarrow I)}_i)$$

(2) - (3) Во второй и третьей частях показано, как полученный результат можно использовать для zero-shot предсказаний на своём датасете. То, почему это названо созданием линейного классификатора можно представить, если принять  $T_1 ... T_n$ за наборы весов нейронов линейного слоя на N нейронов, а представление $I_1$ за вход для классификатора.

<img src="./data/perf_clip.png" alt="Drawing" style="width: 400px;"/>

### Особенности и ограничения CLIP

* Датасет для обучения 400mil пар картинка + текстовое описание
* По данным авторов, версия Zero-Shot CLIP, упомянутая в оригинальной статье, достигает near SOTA (SOTA как правило получено при помощи supervised техник) результатов на большинстве датасетов в supervised задачах. Для того, чтобы приблизиться к SOTA, нужно всего лишь х1000 времени и данных, что, однако __"infeasible to train with current hardware"__
* Модель сталкивается с трудностями генерализации, когда видит что-то, чего не было в обучающей выборке __"While zero-shot CLIP generalizes well to many natural image distributions as investigated in Section 3.3, we’ve observed that zero-shot CLIP still generalizes poorly to data that is truly out-of-distribution for it. ... CLIP learns a high quality semantic OCR representation that performs well on digitally rendered text, which is common in its pre-training dataset, as evidenced by performance on Rendered SST2. However, CLIP only achieves 88% accuracy on the handwritten digits of MNIST. An embarrassingly simple baseline of logistic regression on raw pixels outperforms zero-shot CLIP."__
* Модель не подходит для генерации текстового описания изображения и получила некоторый Social Bias из обучающей выборки. __"CLIP is trained on text paired with images on the internet.These image-text pairs are unfiltered and uncurated and result in CLIP models learning many social biases."__

### Где может быть использован в LDM

* Генерация представления для текста (text-encoder)
* Генерация представления для изображения (image-encoder)
* Ранжирование изображений (DALLE использует CLIP также для упорядочивания сгенерированных изображений перед тем, как отдать их пользователю)

## Что-то для сжатия изображений
![](./data/шакал.jpeg)


Пространство пикселей является пространством очень высокой размерности (каждый пиксель == измерение), в нём очень долго и затратно работать, при этом достаточно малое число пикселей несет действительно важную информацию

Метод (может варьироваться в разных имплементациях LDM, как и почти любой другой компонент): строим отображение в пространство меньшей размерности, в котором, тем не менее, изображение не лишается своих свойств - представление в новом пространстве всё ещё является изображением в привычном для нас понимании этого слова. Таким образом процедура представляет собой получение скетча, который сохраняет в себе максимум информации из входа

Характерным моментом при обучении является подбор размерности, нужно соблюсти баланс между желанием максимального сжатия/ускорения/удешевления расчетов и сохранением детализации в получаемом пространстве

Ниже приведен график, на котором отражена динамика обучения моделей с различным значением downsampling factor, который представляет собой следующее отношение:

$$x ∈ R^{H×W×3};   z ∈ R^{h×w×c}$$


$$f = H/h = W/w$$

![](./data/spatial_downsampling.png)

## Способ задать направление для генерации

### Внимание-внимание!
Слайды позаимствованы из [презентации](https://scholar.harvard.edu/binxuw/classes/machine-learning-scratch/materials/stable-diffusion-scratch)

![](./data/slide_1.png)
![](./data/slide_2.png)
![](./data/slide_3.png)
![](./data/slide_4.png)
![](./data/slide_5.png)
![](./data/slide_6.png)
![](./data/slide_7.png)
![](./data/slide_8.png)
![](./data/slide_9.png)
![](./data/slide_10.png)

## Ещё немного практики

In [ ]:
!pip install diffusers

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler, UNet2DModel
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [ ]:
dataset = torchvision.datasets.MNIST(root="mnist/", train=True, download=True, transform=torchvision.transforms.ToTensor())

In [ ]:
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2')

In [ ]:
class ClassConditionedUnet(nn.Module):
  def __init__(self, num_classes=10, class_emb_size=4):
    super().__init__()

    # The embedding layer will map the class label to a vector of size class_emb_size
    self.class_emb = nn.Embedding(num_classes, class_emb_size)

    # Self.model is an unconditional UNet with extra input channels to accept the conditioning information (the class embedding)
    self.model = UNet2DModel(
        sample_size=28,           # the target image resolution
        in_channels=1 + class_emb_size, # Additional input channels for class cond.
        out_channels=1,           # the number of output channels
        layers_per_block=2,       # how many ResNet layers to use per UNet block
        block_out_channels=(32, 64, 64),
        down_block_types=(
            "DownBlock2D",        # a regular ResNet downsampling block
            "AttnDownBlock2D",    # a ResNet downsampling block with spatial self-attention
            "AttnDownBlock2D",
        ),
        up_block_types=(
            "AttnUpBlock2D",
            "AttnUpBlock2D",      # a ResNet upsampling block with spatial self-attention
            "UpBlock2D",          # a regular ResNet upsampling block
          ),
    )

  # Our forward method now takes the class labels as an additional argument
  def forward(self, x, t, class_labels):
    # Shape of x:
    bs, ch, w, h = x.shape

    # class conditioning in right shape to add as additional input channels
    class_cond = self.class_emb(class_labels) # Map to embedding dimension
    class_cond = class_cond.view(bs, class_cond.shape[1], 1, 1).expand(bs, class_cond.shape[1], w, h)
    # x is shape (bs, 1, 28, 28) and class_cond is now (bs, 4, 28, 28)

    # Net input is now x and class cond concatenated together along dimension 1
    net_input = torch.cat((x, class_cond), 1) # (bs, 5, 28, 28)

    # Feed this to the UNet alongside the timestep and return the prediction
    return self.model(net_input, t).sample # (bs, 1, 28, 28)

In [ ]:
# Redefining the dataloader to set the batch size higher than the demo of 8
train_dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# How many runs through the data should we do?
n_epochs = 10

# Our network
net = ClassConditionedUnet().to('cuda')

# Our loss function
loss_fn = nn.MSELoss()

# The optimizer
opt = torch.optim.Adam(net.parameters(), lr=1e-3)

# Keeping a record of the losses for later viewing
losses = []

# The training loop
for epoch in range(n_epochs):
    for x, y in tqdm(train_dataloader):

        # Get some data and prepare the corrupted version
        x = x.to('cuda') * 2 - 1 # Data on the GPU (mapped to (-1, 1))
        y = y.to('cuda')
        noise = torch.randn_like(x)
        timesteps = torch.randint(0, 999, (x.shape[0],)).long().to('cuda')
        noisy_x = noise_scheduler.add_noise(x, noise, timesteps)

        # Get the model prediction
        pred = net(noisy_x, timesteps, y) # Note that we pass in the labels y

        # Calculate the loss
        loss = loss_fn(pred, noise) # How close is the output to the noise

        # Backprop and update the params:
        opt.zero_grad()
        loss.backward()
        opt.step()

        # Store the loss for later
        losses.append(loss.item())

    # Print out the average of the last 100 loss values to get an idea of progress:
    avg_loss = sum(losses[-100:])/100
    print(f'Finished epoch {epoch}. Average of the last 100 loss values: {avg_loss:05f}')

# View the loss curve
plt.plot(losses)

In [ ]:
# Prepare random x to start from, plus some desired labels y
x = torch.randn(80, 1, 28, 28).to('cuda')
y = torch.tensor([[i]*8 for i in range(10)]).flatten().to('cuda')

# Sampling loop
for i, t in tqdm(enumerate(noise_scheduler.timesteps)):

    # Get model pred
    with torch.no_grad():
        residual = net(x, t, y)  # Again, note that we pass in our labels y

    # Update sample with step
    x = noise_scheduler.step(residual, t, x).prev_sample

# Show the results
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.imshow(torchvision.utils.make_grid(x.detach().cpu().clip(-1, 1), nrow=8)[0], cmap='Greys')

### Если очень хочется прикоснуться к прекрасному

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from torch import autocast
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    use_auth_token=True
).to("cuda")

In [ ]:
prompt = "LOTR landscape cinematic 4k hires"
image = pipe(prompt)

In [ ]:
image.images[0]